In [2]:
import requests
import time
import pandas as pd
import numpy as np
import psycopg2
import sqlalchemy

from dotenv import load_dotenv
import os

load_dotenv()

True

In [3]:
host = os.getenv('HOST')
port = os.getenv('PORT')
user = os.getenv('USER')
password = os.getenv('PASSWORD')
database= os.getenv('DATABASE')

In [3]:
#count  der Messwerte

# Verbindung zur Datenbank herstellen
connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
print("Verbindung zur Datenbank erfolgreich hergestellt.")

# Einen Cursor erstellen, um SQL-Abfragen auszuführen
cursor = connection.cursor()

# SQL SELECT-Abfrage
select_query = "SELECT * FROM value;"

# Abfrage ausführen
cursor.execute(select_query)

# Ergebnis abrufen
rows = cursor.fetchall()

# Ergebnis in ein Pandas DataFrame umwandeln
df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])

# Ergebnis anzeigen
print(df)

# Cursor und Verbindung schließen
cursor.close()
connection.close()


Verbindung zur Datenbank erfolgreich hergestellt.
       index                time  temperature  luftfeuchtigkeit   pm10  pm2_5  \
0          1 2024-05-15 16:32:38        18.30             99.90   3.40   3.10   
1          2 2024-05-15 16:32:39        17.90             75.63   9.45   8.20   
2          3 2024-05-15 16:32:40        22.90             55.90   3.23   2.93   
3          4 2024-05-15 16:32:40        17.24             69.30  10.77   3.38   
4          5 2024-05-15 16:32:41        26.30             30.70   0.00   0.00   
...      ...                 ...          ...               ...    ...    ...   
11515  11516 2024-05-26 12:40:04        17.72             69.81  10.77   3.38   
11516  11517 2024-05-26 12:40:04        17.60             71.10   0.00   0.00   
11517  11518 2024-05-26 12:40:05        39.10             92.00   2.10   1.70   
11518  11519 2024-05-26 12:40:05        22.78              0.00   0.00   0.00   
11519  11520 2024-05-26 12:40:06        27.35             3

In [4]:
senseid_counts = df['senseid_fk'].value_counts()

# Ergebnis anzeigen
print(senseid_counts)

senseid_fk
5ae4a726223bd80019a367a6    1440
5dde9523ba7944001da4150e    1440
5c08379b1c28f9001a3f580c    1440
5b4d11485dc1ec001b5452c7    1440
65e8d93acbf5700007f920ca    1440
590e0b0a51d3460011c725c4    1440
605f498077a88b001bba3dc0    1440
64722d1c9be0580007f776d9    1440
Name: count, dtype: int64


### Messwerte der einzelnen Stationen in ein Dataframe speichern

In [5]:
#Messstationen definieren
boxes_dic = {
    "Mersch Luxembourg" : "5ae4a726223bd80019a367a6",
    "Ettlingen Deutschland" : "5dde9523ba7944001da4150e",
    "Stuttgart Deutschland" : "5c08379b1c28f9001a3f580c",
    "München Deutschland" : "5b4d11485dc1ec001b5452c7",
    "Berlin Deutschland" : "65e8d93acbf5700007f920ca",
    "Hamburg Deutschland" : "590e0b0a51d3460011c725c4",
    "Dortmund Deutschland" : "605f498077a88b001bba3dc0",
    "Jena Deutschland" : "64722d1c9be0580007f776d9"
}

In [6]:
# Erstelle ein leeres DataFrame mit Spaltennamen
df_messwerte = pd.DataFrame(columns=["time", "temperature","luftfeuchtigkeit", "pm10", "pm2_5", "senseid_fk"])
df_id = pd.DataFrame(columns=["temperature_id","luftfeuchtigkeit_id", "pm10_id", "pm2_5_id", "senseid_fk"])

### Alle SensorID holen von den Senseboxen

In [7]:

new_data = []  # A list to collect new data
for box_id, box_value in boxes_dic.items():
    
    try:
        response = requests.get(f'https://api.opensensemap.org/boxes/{box_value}?format=json')
        box = response.json() 

        temperature_value = None
        pm10_value = None
        pm25_value = None
        rel_Luftfeuchte_value = None

        # Traverse through the 'sensors' list
        for sensor in box.get('sensors', []):
            # Check if the sensor is for Temperature
            if sensor.get('title') == 'Temperatur':
                # Access the temperature value
                temperature_value = sensor.get('_id', {})
                

            if sensor.get('title') == 'PM10':
                # Access the PM10 value
                pm10_value = sensor.get('_id', {})

            if sensor.get('title') == 'PM2.5':
                # Access the PM2.5 value
                pm25_value = sensor.get('_id', {})

            if sensor.get('title') == 'rel. Luftfeuchte':
                # Access the relative humidity value
                rel_Luftfeuchte_value = sensor.get('_id', {})
                
                break  # Exit the loop once the temperature is found
            # Append new data to the list
        new_data.append({"temperature_id": temperature_value,
                        "luftfeuchtigkeit_id": rel_Luftfeuchte_value, "pm10_id": pm10_value, "pm2_5_id": pm25_value,
                        "senseid_fk": box_value})
    except:
        print("FUCK YOU SENSEBOX NO WORKING TODAY" + box_value)

new_df = pd.DataFrame(new_data)
    
    # Append the new DataFrame to the existing DataFrame
df_id = pd.concat([df_id, new_df], ignore_index=True)

In [8]:
df_id

,temperature_id,luftfeuchtigkeit_id,pm10_id,pm2_5_id,senseid_fk
0,5ae4a726223bd80019a367a8,5ae4a726223bd80019a367a7,5ae4a726223bd80019a367aa,5ae4a726223bd80019a367a9,5ae4a726223bd80019a367a6
1,5dde9523ba7944001da41511,5dde9523ba7944001da41510,5dde9523ba7944001da41513,5dde9523ba7944001da41512,5dde9523ba7944001da4150e
2,5c08379b1c28f9001a3f580e,5c08379b1c28f9001a3f580d,5c08379b1c28f9001a3f5810,5c08379b1c28f9001a3f580f,5c08379b1c28f9001a3f580c
3,5b4d11485dc1ec001b5452ca,5b4d11485dc1ec001b5452c9,5b4d11485dc1ec001b5452cc,5b4d11485dc1ec001b5452cb,5b4d11485dc1ec001b5452c7
4,65e8d93acbf5700007f920cb,65e8d93acbf5700007f920cc,None,None,65e8d93acbf5700007f920ca
5,592469ca51d34600119bf21f,592469ca51d34600119bf220,590e0b0a51d3460011c725c5,590e0b0a51d3460011c725c6,590e0b0a51d3460011c725c4
6,605f49bd77a88b001bba595a,None,None,None,605f498077a88b001bba3dc0
7,64722d1c9be0580007f776dc,64722d1c9be0580007f776db,64722d1c9be0580007f776de,64722d1c9be0580007f776dd,64722d1c9be0580007f776d9


### Historie der letzten 10000 Daten

In [43]:
df_messwerte = pd.DataFrame(columns=["time", "temperature", "luftfeuchtigkeit", "pm10", "pm2_5", "senseid_fk"])

# Erstelle eine Liste, um die Dictionaries zu speichern
list_of_dicts = []

# Iteriere über jede Zeile im DataFrame, um die Liste der Dictionaries zu erstellen
for _, row in df_id.iterrows():
    senseid_fk = row['senseid_fk']
    sensors = {
        'temperature': row['temperature_id'],
        'luftfeuchtigkeit': row['luftfeuchtigkeit_id'],
        'pm10': row['pm10_id'],
        'pm2_5': row['pm2_5_id']
    }
    sensor_dict = {senseid_fk: sensors}
    list_of_dicts.append(sensor_dict)

# Iteriere über jede Box im Dictionary
for box in list_of_dicts:
    for senseid_fk, sensors in box.items():
        df_box = pd.DataFrame()  # Initialisiere den DataFrame für die aktuelle Box
        
        for sensor_type, sensor_id in sensors.items():
            if sensor_id is not None:  # Überprüfen, ob der Sensor vorhanden ist
                response = requests.get(f'https://api.opensensemap.org/boxes/{senseid_fk}/data/{sensor_id}?from-date=2024-05-01T00:00:00.000Z&to-date=2024-05-24T20:44:01.434Z&download=true&format=json')
                data = response.json()
                
                if data:  # Überprüfen, ob Daten vorhanden sind
                    temp_df = pd.DataFrame(data)
                    temp_df['senseid_fk'] = senseid_fk
                    temp_df = temp_df.rename(columns={'createdAt': 'time', 'value': sensor_type})
                    
                    if df_box.empty:
                        df_box = temp_df[['time', sensor_type, 'senseid_fk']]
                    else:
                        df_box = pd.merge(df_box, temp_df[['time', sensor_type]], on='time', how='outer')
        
        # Zusammenführen der aktuellen Box-Daten mit den bisherigen Daten
        df_messwerte = pd.concat([df_messwerte, df_box], ignore_index=True)

df_messwerte_cleaned = df_messwerte.dropna(subset=['senseid_fk'])


In [44]:
df_messwerte_cleaned['time'] = df_messwerte_cleaned['time'].str[:-1]  # Entfernt das 'Z' am Ende
df_messwerte_cleaned['time'] = pd.to_datetime(df_messwerte_cleaned['time'], format='%Y-%m-%dT%H:%M:%S.%f')


C:\Users\conta\AppData\Local\Temp\ipykernel_7468\3397160356.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_messwerte_cleaned['time'] = df_messwerte_cleaned['time'].str[:-1]  # Entfernt das 'Z' am Ende
C:\Users\conta\AppData\Local\Temp\ipykernel_7468\3397160356.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_messwerte_cleaned['time'] = pd.to_datetime(df_messwerte_cleaned['time'], format='%Y-%m-%dT%H:%M:%S.%f')


In [59]:
df_messwerte_cleaned['time'] = pd.to_datetime(df_messwerte_cleaned['time'])

C:\Users\conta\AppData\Local\Temp\ipykernel_7468\4174200063.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_messwerte_cleaned['time'] = pd.to_datetime(df_messwerte_cleaned['time'])


In [60]:
df_messwerte_cleaned.dtypes

time                datetime64[ns]
temperature                 object
luftfeuchtigkeit            object
pm10                        object
pm2_5                       object
senseid_fk                  object
dtype: object

In [61]:
df_messwerte_cleaned

,time,temperature,luftfeuchtigkeit,pm10,pm2_5,senseid_fk
0,2024-05-07 16:39:12,21.90,84.70,4.07,3.67,5ae4a726223bd80019a367a6
1,2024-05-07 16:41:39,22.00,84.20,3.80,3.50,5ae4a726223bd80019a367a6
2,2024-05-07 16:44:07,22.00,83.40,5.22,4.05,5ae4a726223bd80019a367a6
3,2024-05-07 16:46:35,22.20,82.00,4.33,3.93,5ae4a726223bd80019a367a6
4,2024-05-07 16:49:02,22.10,81.50,4.87,3.50,5ae4a726223bd80019a367a6
...,...,...,...,...,...,...
76908,2024-05-24 20:36:33,15.54,94.76,5.70,3.22,64722d1c9be0580007f776d9
76909,2024-05-24 20:38:14,15.45,95.10,5.18,3.47,64722d1c9be0580007f776d9
76910,2024-05-24 20:39:55,15.44,95.42,5.00,3.70,64722d1c9be0580007f776d9
76911,2024-05-24 20:41:46,15.38,95.29,6.65,4.10,64722d1c9be0580007f776d9


### Input in die Datenbank + Filterung nur Daten vor 15.5. 16.32Uhr

In [62]:
df_messwerte_cleaned.dtypes

time                datetime64[ns]
temperature                 object
luftfeuchtigkeit            object
pm10                        object
pm2_5                       object
senseid_fk                  object
dtype: object

In [63]:
df_filtered = df_messwerte_cleaned[(df_messwerte_cleaned['time'].dt.day<=15) & 
                                   (df_messwerte_cleaned['time'].dt.hour<=16) & 
                                   (df_messwerte_cleaned['time'].dt.minute<32)]

In [64]:
df_filtered

,time,temperature,luftfeuchtigkeit,pm10,pm2_5,senseid_fk
180,2024-05-08 00:02:15,13.50,99.90,11.53,10.33,5ae4a726223bd80019a367a6
181,2024-05-08 00:04:43,13.50,99.90,14.27,11.87,5ae4a726223bd80019a367a6
182,2024-05-08 00:07:10,13.40,99.90,21.90,13.70,5ae4a726223bd80019a367a6
183,2024-05-08 00:09:38,13.40,99.90,21.12,16.40,5ae4a726223bd80019a367a6
184,2024-05-08 00:12:06,13.60,99.90,18.73,14.57,5ae4a726223bd80019a367a6
...,...,...,...,...,...,...
69148,2024-05-15 16:24:14,24.66,28.14,3.15,1.33,64722d1c9be0580007f776d9
69149,2024-05-15 16:25:55,24.56,28.30,2.78,1.50,64722d1c9be0580007f776d9
69150,2024-05-15 16:27:35,24.36,28.50,5.05,1.45,64722d1c9be0580007f776d9
69151,2024-05-15 16:29:17,24.48,28.23,4.15,1.60,64722d1c9be0580007f776d9


In [ ]:
import sqlalchemy

# Define your PostgreSQL connection string
connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"

# Create a SQLAlchemy engine
engine = sqlalchemy.create_engine(connection_string)

try:
    # Connect to the database using psycopg2
    connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
    print("Verbindung zur Datenbank erfolgreich hergestellt.")

    # Now you can proceed with inserting the DataFrame into the PostgreSQL database
    df_filtered.to_sql("value", con=engine, if_exists="append", index=False)
    print("Daten erfolgreich in die Datenbank eingefügt.")
    
except Exception as e:
    print(f"Fehler beim Herstellen der Verbindung zur Datenbank oder Einfügen der Daten: {e}")

### Aktuelle Temperatur

In [11]:
# Assuming df_messwerte is already defined as a DataFrame

new_data = []  # A list to collect new data
for box_id, box_value in boxes_dic.items():
    
    try:
        response = requests.get(f'https://api.opensensemap.org/boxes/{box_value}?format=json')
        box = response.json() 

        temperature_value = None
        pm10_value = None
        pm25_value = None
        rel_Luftfeuchte_value = None

        # Traverse through the 'sensors' list
        for sensor in box.get('sensors', []):
            # Check if the sensor is for Temperature
            if sensor.get('title') == 'Temperatur':
                # Access the temperature value
                temperature_value = sensor.get('lastMeasurement', {}).get('value')
            
            if sensor.get('title') == 'PM10':
                # Access the PM10 value
                pm10_value = sensor.get('lastMeasurement', {}).get('value')

            if sensor.get('title') == 'PM2.5':
                # Access the PM2.5 value
                pm25_value = sensor.get('lastMeasurement', {}).get('value')

            if sensor.get('title') == 'rel. Luftfeuchte':
                # Access the relative humidity value
                rel_Luftfeuchte_value = sensor.get('lastMeasurement', {}).get('value')
                
                break  # Exit the loop once the temperature is found
            
        # Append new data to the list
        new_data.append({"temperature": temperature_value, "time": time.strftime("%Y-%m-%d %H:%M:%S"),
                        "luftfeuchtigkeit": rel_Luftfeuchte_value, "pm10": pm10_value, "pm2_5": pm25_value,
                        "senseid_fk": box_value})
    
    except:
        print("FUCK YOU SENSEBOX NO WORKING TODAY" + box_value)
    
    # Convert the new data list to a DataFrame
new_df = pd.DataFrame(new_data)
    
    # Append the new DataFrame to the existing DataFrame
df_messwerte = pd.concat([df_messwerte, new_df], ignore_index=True)
df_messwerte = df_messwerte.fillna(0)

# Convert specific columns to float64, handling non-numeric values
columns_to_convert = ["temperature", "luftfeuchtigkeit", "pm10", "pm2_5"]
for column in columns_to_convert:
    df_messwerte[column] = pd.to_numeric(df_messwerte[column], errors='coerce')

df_messwerte = df_messwerte.round(2)


None
None
None
None
None
None
None


In [9]:
df_messwerte

,time,temperature,luftfeuchtigkeit,pm10,pm2_5,senseid_fk
0,2024-05-24 23:37:43,0,0,0,0,5ae4a726223bd80019a367a6
1,2024-05-24 23:37:43,0,0,0,0,5dde9523ba7944001da4150e
2,2024-05-24 23:37:43,0,0,0,0,5c08379b1c28f9001a3f580c
3,2024-05-24 23:37:43,0,0,0,0,5b4d11485dc1ec001b5452c7
4,2024-05-24 23:37:43,0,0,0,0,65e8d93acbf5700007f920ca
5,2024-05-24 23:37:43,0,0,0,0,590e0b0a51d3460011c725c4
6,2024-05-24 23:37:44,0,0,0,0,605f498077a88b001bba3dc0
7,2024-05-24 23:37:44,0,0,0,0,64722d1c9be0580007f776d9
8,2024-05-24 23:38:29,None,None,None,None,5ae4a726223bd80019a367a6
9,2024-05-24 23:38:30,None,None,None,None,5dde9523ba7944001da4150e


In [7]:
df_messwerte["time"] = pd.to_datetime(df_messwerte["time"])

In [8]:
df_messwerte.dtypes

time                datetime64[ns]
temperature                float64
luftfeuchtigkeit           float64
pm10                       float64
pm2_5                      float64
senseid_fk                  object
dtype: object

In [9]:
import sqlalchemy

# Define your PostgreSQL connection string
connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"

# Create a SQLAlchemy engine
engine = sqlalchemy.create_engine(connection_string)

try:
    # Connect to the database using psycopg2
    connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
    print("Verbindung zur Datenbank erfolgreich hergestellt.")

    # Now you can proceed with inserting the DataFrame into the PostgreSQL database
    df_messwerte.to_sql("value", con=engine, if_exists="append", index=False)
    print("Daten erfolgreich in die Datenbank eingefügt.")
    
except Exception as e:
    print(f"Fehler beim Herstellen der Verbindung zur Datenbank oder Einfügen der Daten: {e}")


Verbindung zur Datenbank erfolgreich hergestellt.
Daten erfolgreich in die Datenbank eingefügt.


### Zweites DataFrame für Daten der Messstationen erstellen

In [10]:
# Erstelle ein leeres DataFrame mit Spaltennamen
df_stations = pd.DataFrame(columns=["Ort","Land","Lat", "Lon", "Hoehe", "senseID"])

In [11]:
new_data = []  # Eine leere Liste, um neue Daten zu sammeln
for box_id, box_value in boxes_dic.items():
    try:
        response = requests.get(f'https://api.opensensemap.org/boxes/{box_value}?format=json')
        box = response.json() 
    except:
        print("FUCK YOU SENSEBOX NO WORKING TODAY" + box_value)
    temperature_value = None
    lat = None
    lon = None
    hoehe = None
    pm10_value = None
    pm25_value = None
    rel_Luftfeuchte_value = None
    ort_Land = box_id.split()

    # Extrahiere Koordinaten
    current_location = box.get('currentLocation', {})
    if current_location:
        coordinates = current_location.get('coordinates')
        if coordinates:
            lon = coordinates[0]
            lat = coordinates[1]
            height = coordinates[2] if len(coordinates) >= 3 else None

        
    # Füge neue Daten zur Liste hinzu
    new_data.append({"Ort": ort_Land[0], "Land":ort_Land[1], "Lat": lat,"Lon":lon,"Hoehe":height, "senseID": box_value})
    
    # Konvertiere die neue Datenliste in ein DataFrame
new_df = pd.DataFrame(new_data)
    
    # Hänge das neue DataFrame an das bestehende DataFrame an
df_stations = pd.concat([df_stations, new_df], ignore_index=True)

FUCK YOU SENSEBOX NO WORKING TODAY590e0b0a51d3460011c725c4


In [12]:
df_stations

,Ort,Land,Lat,Lon,Hoehe,senseID
0,Mersch,Luxembourg,49.754663,6.105682,250.0,5ae4a726223bd80019a367a6
1,Ettlingen,Deutschland,48.926209,8.374409,4.0,5dde9523ba7944001da4150e
2,Stuttgart,Deutschland,48.779610,9.213723,280.0,5c08379b1c28f9001a3f580c
3,München,Deutschland,48.146546,11.576033,20.0,5b4d11485dc1ec001b5452c7
4,Berlin,Deutschland,52.511270,13.397356,45.0,65e8d93acbf5700007f920ca
5,Hamburg,Deutschland,52.511270,13.397356,45.0,590e0b0a51d3460011c725c4
6,Dortmund,Deutschland,51.497350,7.494606,NaN,605f498077a88b001bba3dc0
7,Jena,Deutschland,50.934561,11.589701,146.0,64722d1c9be0580007f776d9


### Daten von der DB lesen

In [17]:
try:
    # Connect to the database using psycopg2
    connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
    print("Verbindung zur Datenbank erfolgreich hergestellt.")

    # SQL-Abfrage ausführen
    query = "SELECT * FROM value"  
    cursor = connection.cursor()
    cursor.execute(query)
    results = cursor.fetchall()

    # DataFrame erstellen
    df = pd.DataFrame(results, columns=[desc[0] for desc in cursor.description])
    df = df.drop(columns=["index"])
    # Verbindung schließen
    connection.close()
    print("Daten erfolgreich geladen.")
    
except Exception as e:
    print(f"Fehler beim Herstellen der Verbindung zur Datenbank oder Einfügen der Daten: {e}")


Verbindung zur Datenbank erfolgreich hergestellt.
Daten erfolgreich geladen.


In [18]:
df

,time,temperature,luftfeuchtigkeit,pm10,pm2_5,senseid_fk
0,2024-05-15 16:32:38,18.30,99.90,3.40,3.10,5ae4a726223bd80019a367a6
1,2024-05-15 16:32:39,17.90,75.63,9.45,8.20,5dde9523ba7944001da4150e
2,2024-05-15 16:32:40,22.90,55.90,3.23,2.93,5c08379b1c28f9001a3f580c
3,2024-05-15 16:32:40,17.24,69.30,10.77,3.38,5b4d11485dc1ec001b5452c7
4,2024-05-15 16:32:41,26.30,30.70,0.00,0.00,65e8d93acbf5700007f920ca
...,...,...,...,...,...,...
2075,2024-05-17 14:10:03,17.39,70.37,10.77,3.38,5b4d11485dc1ec001b5452c7
2076,2024-05-17 14:10:03,17.60,43.50,0.00,0.00,65e8d93acbf5700007f920ca
2077,2024-05-17 14:10:04,38.90,30.00,2.53,1.73,590e0b0a51d3460011c725c4
2078,2024-05-17 14:10:05,13.50,0.00,0.00,0.00,605f498077a88b001bba3dc0


### plotly / dash (aus Übung)

In [48]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd
import requests
from datetime import datetime

In [ ]:
# Initialize Dash app
app = dash.Dash(__name__)

# Create layout
app.layout = html.Div([
    # Graph component
    dcc.Graph(id='live-graph'),

    # Triggers periodically
    dcc.Interval(
        id='interval-component',
        interval=10_000,
        n_intervals=0
    )
])

# Initialize empty dataframe
df = pd.DataFrame(columns=['timestamp', 'value'])


# Define callback to update graph
@app.callback(
    Output('live-graph', 'figure'),
    [Input('interval-component', 'n_intervals')]
)
def update_graph(n):  # increment from intervals

    # Request API
    box_id = "616dd3698bd2c9001cc6d293"
    url = f"https://api.opensensemap.org/boxes/{box_id}?format=json"
    data = requests.get(url).json()
    value = data['sensors'][2]['lastMeasurement']['value']
    timestamp = datetime.now()

    global df
    df = pd.concat([df, pd.DataFrame([{'timestamp': timestamp, 'value': float(value)}])], ignore_index=True)

    # Plot data
    trace = go.Scatter(x=df['timestamp'], y=df['value'], mode='lines+markers')
    layout = go.Layout(title='Real-time Sensor Data', xaxis=dict(title='Timestamp'), yaxis=dict(title='Value'))
    fig = go.Figure(data=[trace], layout=layout)
    return fig


if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

### Random Forest Modell trainieren

In [19]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error


# 1. Convert the time column to datetime objects
df['time'] = pd.to_datetime(df['time'])

# 2. Feature Engineering with Lags
lags = [1, 2, 3]  # Lags for 10, 20, and 30 minutes
for lag in lags:
    df[f'Temperatur_lag_{lag*10}'] = df.groupby('senseid_fk')['temperature'].shift(lag)

# 3. Create target variables for each prediction horizon
for minutes in [10, 20, 30]:
    shift = minutes // 10  # Calculate the shift based on 10-minute intervals
    df[f'Temperatur_in_{minutes}_Minuten'] = df.groupby('senseid_fk')['temperature'].shift(-shift)

# 4. Remove rows with missing values (introduced by lagging and shifting)
df.dropna(inplace=True)


In [20]:
# 5. Define target variables and create a dictionary for models
target_variables = ['Temperatur_in_10_Minuten', 'Temperatur_in_20_Minuten', 'Temperatur_in_30_Minuten']
models = {}

# 6. Train and evaluate models for each target variable
for target in target_variables:
    # 6.1. Select features and target variable
    X = df.drop(target_variables + ["time", "senseid_fk"], axis=1)
    y = df[target]

    # 6.2. Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # 6.3. Create and train the Random Forest model
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # 6.4. Make predictions and evaluate
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    print(f"\n{target}:")
    print(f"  Mean Squared Error: {mse}")
    print(f"  Mean Absolute Error: {mae}")

    # 6.5. Store the trained model
    models[target] = model


Temperatur_in_10_Minuten:
  Mean Squared Error: 0.45367572713415544
  Mean Absolute Error: 0.2834032262135375

Temperatur_in_20_Minuten:
  Mean Squared Error: 0.658470633013489
  Mean Absolute Error: 0.3806042174652441

Temperatur_in_30_Minuten:
  Mean Squared Error: 0.8626496319008645
  Mean Absolute Error: 0.4876920320645823


Kartendarstellung

In [ ]:
import plotly.express as px

# Create a scatter mapbox plot
fig = px.scatter_mapbox(df_stations, lat="Lat", lon="Lon", hover_name="Ort", hover_data=["Land", "Hoehe"],
                        color_discrete_sequence=["blue"], zoom=3, height=300)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

# Add the map to your layout
html.Div([
    dcc.Graph(figure=fig)
])

Ein paar Kennzahlen


In [ ]:
# Calculate KPIs
average_temperature = df['temperature'].mean()
max_pm10 = df['pm10'].max()
min_pm2_5 = df['pm2_5'].min()

# Create KPI components
html.Div([
    html.H4("KPIs"),
    html.P(f"Durchschnittstemperatur: {average_temperature:.2f}°C"),
    html.P(f"Maximales PM10: {max_pm10}"),
    html.P(f"Minimales PM2.5: {min_pm2_5}")
])


Zeitreihen

In [ ]:
# Beispiel für Aggregation über Stunden und Visualisierung als Linienplot
hourly_aggregated = df.groupby(df['time'].dt.hour)['temperature'].mean()

fig = px.line(hourly_aggregated, x=hourly_aggregated.index, y=hourly_aggregated.values, 
              labels={'x':'Stunde des Tages', 'y':'Durchschnittstemperatur (°C)'})
